In [4]:
# Link Logistic regression decision back to the originalfeatures
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from pca_utils import save_pca_outputs

# Step 2: Load the dataset
cancer = load_breast_cancer()
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = pd.Series(cancer.target, name='target') 

# Step 3: Standardize the features 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Reduce dimensions to 2 PCA components
pca = PCA(n_components=2)  
X_pca = pca.fit_transform(X_scaled)

# Step 5: Train logistic regression
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

# step 6: Initialize and train logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Step 7: Logistic regression coefficient (importance of PC)
logreg_coefs = logreg.coef_[0]   # shape =[2]

# Step 8: PCA Loadings (importance of original feature for each PC)
loadings = pd.DataFrame(pca.components_.T, index=X.columns,
                        columns=['PC1', 'PC2'])

# Step 9: Calculate the contribution of original features to logistics regression
feature_contribution = loadings.values @ logreg_coefs
feature_contribution = pd.Series(feature_contribution, index=X.columns).sort_values(ascending=False)

# Step 10: Display top 10 contributing features
print("Top 10 original feature contributing to logistic regreeion decision:")
top10_features=feature_contribution.head(10)

top10_features.to_excel(
    r"C:\Users\jeffr\principal_component_analysis_project\model_outputs\top10_features.xlsx",
    sheet_name="Top10_Features",
    index=True
)
print(top10_features)




Top 10 original feature contributing to logistic regreeion decision:
mean fractal dimension     0.283592
smoothness error           0.200998
symmetry error             0.121736
fractal dimension error    0.109608
texture error              0.066343
worst fractal dimension    0.045662
worst smoothness          -0.062648
mean symmetry             -0.062820
mean smoothness           -0.076459
compactness error         -0.079758
dtype: float64
